# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f45fd237220>
├── 'a' --> tensor([[ 1.7912, -1.6728, -0.5915],
│                   [-0.4269, -1.1174, -0.4812]])
└── 'x' --> <FastTreeValue 0x7f45fd289850>
    └── 'c' --> tensor([[-1.6105, -0.0648,  1.4361, -0.2247],
                        [-1.3015,  1.0213, -0.4828,  1.3089],
                        [ 1.1834,  0.3474,  0.2893, -0.5656]])

In [4]:
t.a

tensor([[ 1.7912, -1.6728, -0.5915],
        [-0.4269, -1.1174, -0.4812]])

In [5]:
%timeit t.a

63.3 ns ± 0.0469 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f45fd237220>
├── 'a' --> tensor([[-0.4441, -0.9076,  1.5155],
│                   [ 1.6735, -0.1224,  0.0696]])
└── 'x' --> <FastTreeValue 0x7f45fd289850>
    └── 'c' --> tensor([[-1.6105, -0.0648,  1.4361, -0.2247],
                        [-1.3015,  1.0213, -0.4828,  1.3089],
                        [ 1.1834,  0.3474,  0.2893, -0.5656]])

In [7]:
%timeit t.a = new_value

63.4 ns ± 0.0293 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.7912, -1.6728, -0.5915],
               [-0.4269, -1.1174, -0.4812]]),
    x: Batch(
           c: tensor([[-1.6105, -0.0648,  1.4361, -0.2247],
                      [-1.3015,  1.0213, -0.4828,  1.3089],
                      [ 1.1834,  0.3474,  0.2893, -0.5656]]),
       ),
)

In [10]:
b.a

tensor([[ 1.7912, -1.6728, -0.5915],
        [-0.4269, -1.1174, -0.4812]])

In [11]:
%timeit b.a

53.9 ns ± 0.0225 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.6380, -0.0588,  0.1528],
               [ 1.0234, -0.8344, -1.7921]]),
    x: Batch(
           c: tensor([[-1.6105, -0.0648,  1.4361, -0.2247],
                      [-1.3015,  1.0213, -0.4828,  1.3089],
                      [ 1.1834,  0.3474,  0.2893, -0.5656]]),
       ),
)

In [13]:
%timeit b.a = new_value

488 ns ± 0.102 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

827 ns ± 0.0789 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.6 µs ± 19 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

138 µs ± 239 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

139 µs ± 445 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f4543444d90>
├── 'a' --> tensor([[[ 1.7912, -1.6728, -0.5915],
│                    [-0.4269, -1.1174, -0.4812]],
│           
│                   [[ 1.7912, -1.6728, -0.5915],
│                    [-0.4269, -1.1174, -0.4812]],
│           
│                   [[ 1.7912, -1.6728, -0.5915],
│                    [-0.4269, -1.1174, -0.4812]],
│           
│                   [[ 1.7912, -1.6728, -0.5915],
│                    [-0.4269, -1.1174, -0.4812]],
│           
│                   [[ 1.7912, -1.6728, -0.5915],
│                    [-0.4269, -1.1174, -0.4812]],
│           
│                   [[ 1.7912, -1.6728, -0.5915],
│                    [-0.4269, -1.1174, -0.4812]],
│           
│                   [[ 1.7912, -1.6728, -0.5915],
│                    [-0.4269, -1.1174, -0.4812]],
│           
│                   [[ 1.7912, -1.6728, -0.5915],
│                    [-0.4269, -1.1174, -0.4812]]])
└── 'x' --> <FastTreeValue 0x7f4543412fa0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33.3 µs ± 13.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f4543a75c40>
├── 'a' --> tensor([[ 1.7912, -1.6728, -0.5915],
│                   [-0.4269, -1.1174, -0.4812],
│                   [ 1.7912, -1.6728, -0.5915],
│                   [-0.4269, -1.1174, -0.4812],
│                   [ 1.7912, -1.6728, -0.5915],
│                   [-0.4269, -1.1174, -0.4812],
│                   [ 1.7912, -1.6728, -0.5915],
│                   [-0.4269, -1.1174, -0.4812],
│                   [ 1.7912, -1.6728, -0.5915],
│                   [-0.4269, -1.1174, -0.4812],
│                   [ 1.7912, -1.6728, -0.5915],
│                   [-0.4269, -1.1174, -0.4812],
│                   [ 1.7912, -1.6728, -0.5915],
│                   [-0.4269, -1.1174, -0.4812],
│                   [ 1.7912, -1.6728, -0.5915],
│                   [-0.4269, -1.1174, -0.4812]])
└── 'x' --> <FastTreeValue 0x7f4543442f40>
    └── 'c' --> tensor([[-1.6105, -0.0648,  1.4361, -0.2247],
                        [-1.3015,  1.0213, -0.4828,  1.3089],
                 

In [23]:
%timeit t_cat(trees)

30.9 µs ± 38.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

60.2 µs ± 253 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 1.7912, -1.6728, -0.5915],
                [-0.4269, -1.1174, -0.4812]],
       
               [[ 1.7912, -1.6728, -0.5915],
                [-0.4269, -1.1174, -0.4812]],
       
               [[ 1.7912, -1.6728, -0.5915],
                [-0.4269, -1.1174, -0.4812]],
       
               [[ 1.7912, -1.6728, -0.5915],
                [-0.4269, -1.1174, -0.4812]],
       
               [[ 1.7912, -1.6728, -0.5915],
                [-0.4269, -1.1174, -0.4812]],
       
               [[ 1.7912, -1.6728, -0.5915],
                [-0.4269, -1.1174, -0.4812]],
       
               [[ 1.7912, -1.6728, -0.5915],
                [-0.4269, -1.1174, -0.4812]],
       
               [[ 1.7912, -1.6728, -0.5915],
                [-0.4269, -1.1174, -0.4812]]]),
    x: Batch(
           c: tensor([[[-1.6105, -0.0648,  1.4361, -0.2247],
                       [-1.3015,  1.0213, -0.4828,  1.3089],
                       [ 1.1834,  0.3474,  0.2893, -0.5656]],
         

In [26]:
%timeit Batch.stack(batches)

78.9 µs ± 131 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 1.7912, -1.6728, -0.5915],
               [-0.4269, -1.1174, -0.4812],
               [ 1.7912, -1.6728, -0.5915],
               [-0.4269, -1.1174, -0.4812],
               [ 1.7912, -1.6728, -0.5915],
               [-0.4269, -1.1174, -0.4812],
               [ 1.7912, -1.6728, -0.5915],
               [-0.4269, -1.1174, -0.4812],
               [ 1.7912, -1.6728, -0.5915],
               [-0.4269, -1.1174, -0.4812],
               [ 1.7912, -1.6728, -0.5915],
               [-0.4269, -1.1174, -0.4812],
               [ 1.7912, -1.6728, -0.5915],
               [-0.4269, -1.1174, -0.4812],
               [ 1.7912, -1.6728, -0.5915],
               [-0.4269, -1.1174, -0.4812]]),
    x: Batch(
           c: tensor([[-1.6105, -0.0648,  1.4361, -0.2247],
                      [-1.3015,  1.0213, -0.4828,  1.3089],
                      [ 1.1834,  0.3474,  0.2893, -0.5656],
                      [-1.6105, -0.0648,  1.4361, -0.2247],
                      [-1.3015,  

In [28]:
%timeit Batch.cat(batches)

145 µs ± 345 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

310 µs ± 2.49 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
